In [ ]:
import mediapipe as mp
import cv2
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 

In [ ]:
DATA_DIR = '../data/asl_alphabet_train/asl_alphabet_train'
OUTPUT_FILE = '../data/asl_landmarks_train.csv'